Цель: реализовать рекомендательную систему на основе User-Item коллаборативной фильтрации с использованием данных о взаимодействиях пользователей с товарами (например, оценки, покупки).

In [56]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

1.	Подготовка данных 
Использовать набор данных с информацией о пользователях, товарах и их оценках: MovieLens или другой похожий набор данных (Датасет MovieLens содержит 100,000 оценок от 943 пользователей для 1682 фильмов).\
Датасеты находятся по следующему адресу:\
https://grouplens.org/datasets/movielens/ \
Желательно выбирать небольшие по объему датасеты, и в том числе те, которые содержат дополнительную информацию (например, пол человека).
Провести предварительную обработку данных, такую как удаление пропущенных значений и фильтрация неактивных пользователей или товаров.

In [57]:
movies = pd.read_csv('data/ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [58]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [59]:
links = pd.read_csv('data/ml-latest-small/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [60]:
tags = pd.read_csv('data/ml-latest-small/tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [61]:
links.isna().sum()

movieId    0
imdbId     0
tmdbId     8
dtype: int64

In [62]:
ratings.isna().sum().sum(), movies.isna().sum().sum(), tags.isna().sum().sum()

(0, 0, 0)

In [63]:
ratings['userId'].value_counts()

userId
414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
207      20
442      20
53       20
576      20
595      20
Name: count, Length: 610, dtype: int64

In [64]:
min_movie_ratings = 10
movie_counts = ratings['movieId'].value_counts()
active_movies = movie_counts[movie_counts >= min_movie_ratings].index
ratings = ratings[ratings['movieId'].isin(active_movies)]
ratings.shape

(81116, 4)

In [65]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
tags['timestamp'] = pd.to_datetime(tags['timestamp'], unit='s')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51


In [66]:
genre_dummies = movies['genres'].str.get_dummies(sep='|')
movies = pd.concat([movies, genre_dummies], axis=1)
movies.shape

(9742, 23)

In [67]:
merged = ratings.merge(movies, on="movieId", how="left")
merged.head()

,userId,movieId,rating,timestamp,title,genres,(no genres listed),Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


2.	Реализация User-Item коллаборативной фильтрации \
2.1. Построить матрицу взаимодействий (User-Item), где строки — это пользователи, а столбцы — это товары (например, фильмы), значения — оценки или количество взаимодействий. \
2.2. Реализовать алгоритм коллаборативной фильтрации на основе сходства пользователей или товаров: \
User-based: Находить похожих пользователей на основе их оценок. \
Item-based: Находить похожие товары на основе оценок пользователей. 

In [68]:
user_item_matrix = merged.pivot_table(index='userId', columns='movieId', values='rating')
user_item_matrix.head()

movieId,1,2,3,5,6,7,9,10,11,12,...,166461,166528,166643,168250,168252,174055,176371,177765,179819,187593
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
user_item_matrix.shape

(610, 2269)

In [70]:
def get_top_similar_users(user_id, user_item_matrix, k=5):
    user_vector = user_item_matrix.loc[[user_id]].fillna(0)  
    sim_vector = cosine_similarity(user_vector, user_item_matrix.fillna(0))[0]
    
    sim_series = pd.Series(sim_vector, index=user_item_matrix.index)
    sim_series = sim_series.drop(user_id, errors='ignore')
    top_k = sim_series.nlargest(k)
    
    return top_k

top_users_for_1 = get_top_similar_users(user_id=1, user_item_matrix=user_item_matrix, k=5)

print("Топ-5 пользователей, похожих на user 1:")
top_users_for_1

Топ-5 пользователей, похожих на user 1:


userId
368    0.393441
313    0.386130
91     0.380619
266    0.379847
57     0.374995
dtype: float64

In [71]:
def get_top_similar_items(item_id, user_item_matrix, k=5):
    item_vector = user_item_matrix[[item_id]].fillna(0)  
    sim_vector = cosine_similarity(item_vector.T, user_item_matrix.fillna(0).T)[0]
    
    sim_series = pd.Series(sim_vector, index=user_item_matrix.columns)
    sim_series = sim_series.drop(item_id, errors='ignore')
    top_k = sim_series.nlargest(k)
    
    return top_k

top_users_for_1 = get_top_similar_items(item_id=1, user_item_matrix=user_item_matrix, k=5)

print("Топ-5 фильмов, похожих на movie 1:")
top_users_for_1

Топ-5 фильмов, похожих на movie 1:


movieId
3114    0.572601
480     0.565637
780     0.564262
260     0.557388
356     0.547096
dtype: float64

3.	Рекомендации \
3.1. Для заданного пользователя предсказать оценки для товаров, которые он ещё не оценил, на основе похожих пользователей или товаров.\
3.2. Выдать пользователю список рекомендованных товаров.

In [72]:
def compute_user_similarity(user_item_matrix):
    filled_matrix = user_item_matrix.fillna(0)
    similarity = cosine_similarity(filled_matrix)
    return pd.DataFrame(similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def predict_ratings_for_unrated(user_id, user_item_matrix, user_similarity):
    user_ratings = user_item_matrix.loc[user_id]
    unrated_items = user_ratings[user_ratings.isna()].index
    
    predictions = {}
    for item in unrated_items:
        ratings_for_item = user_item_matrix[item]
        valid_users = ratings_for_item[ratings_for_item.notnull()].index.drop(user_id, errors='ignore')
        sims = user_similarity.loc[user_id, valid_users]
        numerator = (sims * ratings_for_item.loc[valid_users]).sum()
        denominator = sims.abs().sum()
        
        predictions[item] = numerator / denominator
            
    return pd.Series(predictions)

user_similarity = compute_user_similarity(user_item_matrix)
i, k = 1, 5
predicted_ratings = predict_ratings_for_unrated(user_id=i, 
                                                user_item_matrix=user_item_matrix, 
                                                user_similarity=user_similarity)

print(f"Предсказанные рейтинги для неоцененных фильмов (user-based) пользователя {i}:")
print(predicted_ratings.sort_values(ascending=False))
print("\nРекомендуемые фильмы:")
print(predicted_ratings.sort_values(ascending=False).index[:k].tolist())

Предсказанные рейтинги для неоцененных фильмов (user-based) пользователя 1:
92535    4.580763
1178     4.579159
1041     4.505968
3451     4.481526
1217     4.458142
           ...   
3564     1.631375
3268     1.612754
1556     1.553652
3593     1.540229
1760     1.294999
Length: 2058, dtype: float64

Рекомендуемые фильмы:
[92535, 1178, 1041, 3451, 1217]


In [73]:
def compute_item_similarity(user_item_matrix):
    filled_matrix = user_item_matrix.fillna(0)
    similarity = cosine_similarity(filled_matrix.T)
    return pd.DataFrame(similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

def predict_ratings_for_unrated_item_based(user_id, user_item_matrix, item_similarity):
    user_ratings = user_item_matrix.loc[user_id]
    unrated_items = user_ratings[user_ratings.isna()].index
    
    predictions = {}
    for item in unrated_items:
        rated_items = user_ratings[user_ratings.notna()].index
        sims = item_similarity.loc[item, rated_items]
        ratings = user_ratings[rated_items]
        
        numerator = (sims * ratings).sum()
        denominator = sims.abs().sum()
        
        predictions[item] = numerator / denominator if denominator != 0 else np.nan
            
    return pd.Series(predictions)

item_similarity = compute_item_similarity(user_item_matrix)
i, k = 1, 5
predicted_ratings_item_based = predict_ratings_for_unrated_item_based(user_id=i, 
                                                                      user_item_matrix=user_item_matrix, 
                                                                      item_similarity=item_similarity)

print(f"Предсказанные рейтинги для неоцененных фильмов (item-based) пользователя {i}:")
print(predicted_ratings_item_based.sort_values(ascending=False))
print("\nРекомендуемые фильмы:")
print(predicted_ratings_item_based.sort_values(ascending=False).index[:k].tolist())

Предсказанные рейтинги для неоцененных фильмов (item-based) пользователя 1:
177765    4.528042
166461    4.507103
98243     4.495090
116897    4.480123
96821     4.479606
            ...   
810       4.265421
489       4.256242
258       4.247809
169       4.225525
9         4.200826
Length: 2058, dtype: float64

Рекомендуемые фильмы:
[177765, 166461, 98243, 116897, 96821]


4.	Оценка модели \
4.1. Разделить данные на обучающую и тестовую выборки. \
4.2. Использовать метрики качества для оценки модели, такие как RMSE (Root Mean Squared Error) или MAE (Mean Absolute Error). \
4.3. Проанализировать результаты и предложить улучшения. \
Например, варьировать параметры и наблюдать, как изменяются метрики (RMSE, MAE). 

In [42]:
train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=42)

train_matrix = train_df.pivot(index='userId', columns='movieId', values='rating')

def compute_item_similarity(user_item_matrix):
    filled_matrix = user_item_matrix.fillna(0)
    similarity = cosine_similarity(filled_matrix.T)
    return pd.DataFrame(similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

def predict_rating_item_based(user, movie, user_item_matrix, item_similarity):
    user_ratings = user_item_matrix.loc[user]
    rated_items = user_ratings[user_ratings.notna()].index
    if len(rated_items) == 0:
        return np.nan
    sims = item_similarity.loc[movie, rated_items]
    ratings = user_ratings[rated_items]
    numerator = (sims * ratings).sum()
    denominator = sims.abs().sum()
    return numerator / denominator if denominator != 0 else np.nan

def compute_user_similarity(user_item_matrix):
    filled_matrix = user_item_matrix.fillna(0)
    similarity = cosine_similarity(filled_matrix)
    return pd.DataFrame(similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def predict_rating_user_based(user, movie, user_item_matrix, user_similarity):
    ratings_for_item = user_item_matrix[movie]
    valid_users = ratings_for_item[ratings_for_item.notnull()].index.drop(user, errors='ignore')
    if len(valid_users) == 0:
        return np.nan
    sims = user_similarity.loc[user, valid_users]
    numerator = (sims * ratings_for_item.loc[valid_users]).sum()
    denominator = sims.abs().sum()
    return numerator / denominator if denominator != 0 else np.nan

In [43]:
item_similarity = compute_item_similarity(train_matrix)
user_similarity = compute_user_similarity(train_matrix)

item_preds = []
user_preds = []
ensemble_preds = []
actuals = []

for idx, row in test_df.iterrows():
    user = row['userId']
    movie = row['movieId']
    actual = row['rating']
    
    if user not in train_matrix.index or movie not in train_matrix.columns:
        continue
    
    pred_item = predict_rating_item_based(user, movie, train_matrix, item_similarity)
    pred_user = predict_rating_user_based(user, movie, train_matrix, user_similarity)
    
    if np.isnan(pred_item) and np.isnan(pred_user):
        continue

    item_preds.append(pred_item)
    user_preds.append(pred_user)

    if not np.isnan(pred_item) and not np.isnan(pred_user):
        ensemble_pred = (pred_item + pred_user) / 2
    elif not np.isnan(pred_item):
        ensemble_pred = pred_item
    else:
        ensemble_pred = pred_user
    ensemble_preds.append(ensemble_pred)
    
    actuals.append(actual)


rmse_item = np.sqrt(mean_squared_error(actuals, item_preds))
mae_item = mean_absolute_error(actuals, item_preds)

rmse_user = np.sqrt(mean_squared_error(actuals, user_preds))
mae_user = mean_absolute_error(actuals, user_preds)

rmse_ensemble = np.sqrt(mean_squared_error(actuals, ensemble_preds))
mae_ensemble = mean_absolute_error(actuals, ensemble_preds)

print("Метрики для Item-Based CF:")
print(f"  RMSE: {rmse_item:.3f}, MAE: {mae_item:.3f}")

print("\nМетрики для User-Based CF:")
print(f"  RMSE: {rmse_user:.3f}, MAE: {mae_user:.3f}")

print("\nМетрики для Ensemble (усреднение):")
print(f"  RMSE: {rmse_ensemble:.3f}, MAE: {mae_ensemble:.3f}")

Метрики для Item-Based CF:
  RMSE: 0.909, MAE: 0.700

Метрики для User-Based CF:
  RMSE: 0.942, MAE: 0.730

Метрики для Ensemble (усреднение):
  RMSE: 0.877, MAE: 0.677


5.	Проанализировать недостатки User-based подхода, такие как холодный старт и проблемы со слишком редкими данными. 

In [44]:
data = {
    'item1': {1: 4.0, 2: 3.0, 3: np.nan, 4: np.nan},
    'item2': {1: 5.0, 2: np.nan, 3: 4.0, 4: np.nan},
    'item3': {1: np.nan, 2: 2.0, 3: np.nan, 4: np.nan},
    'item4': {1: np.nan, 2: np.nan, 3: np.nan, 4: np.nan}
}
user_item_matrix = pd.DataFrame(data)
user_item_matrix

,item1,item2,item3,item4
1,4.0,5.0,NaN,NaN
2,3.0,NaN,2.0,NaN
3,NaN,4.0,NaN,NaN
4,NaN,NaN,NaN,NaN


In [45]:
user_similarity = compute_user_similarity(user_item_matrix)
user_similarity

,1,2,3,4
1,1.000000,0.519778,0.780869,0.0
2,0.519778,1.000000,0.000000,0.0
3,0.780869,0.000000,1.000000,0.0
4,0.000000,0.000000,0.000000,0.0


In [46]:
pred_104 = predict_rating_user_based(4, 'item1', user_item_matrix, user_similarity)
print(f"Пользователь 4 (холодный старт): предсказанный рейтинг для item1 = {pred_104}")

Пользователь 4 (холодный старт): предсказанный рейтинг для item1 = nan


In [47]:
pred_before = predict_rating_user_based(3, 'item1', user_item_matrix, user_similarity)
print(f"Исходное предсказание для пользователя 3 с редкими данными на item1:\n{pred_before:.3f}")

user_item_matrix.loc[1, 'item1'] = 2

user_similarity_new = compute_user_similarity(user_item_matrix)
pred_after = predict_rating_user_based(3, 'item1', user_item_matrix, user_similarity_new)
print(f"Новое предсказание для пользователя 3 на item1 после изменения одной оценки в данных:\n{pred_after:.3f}")

Исходное предсказание для пользователя 3 с редкими данными на item1:
4.000
Новое предсказание для пользователя 3 на item1 после изменения одной оценки в данных:
2.000


6.	Перейти к Item-based коллаборативной фильтрации, сравнить результаты. 

In [48]:
data = {
    'item1': {1: 4.0, 2: 3.0, 3: np.nan, 4: np.nan},
    'item2': {1: 5.0, 2: np.nan, 3: 4.0, 4: np.nan},
    'item3': {1: np.nan, 2: 2.0, 3: np.nan, 4: np.nan},
    'item4': {1: np.nan, 2: np.nan, 3: np.nan, 4: np.nan}
}
user_item_matrix = pd.DataFrame(data)
user_item_matrix

,item1,item2,item3,item4
1,4.0,5.0,NaN,NaN
2,3.0,NaN,2.0,NaN
3,NaN,4.0,NaN,NaN
4,NaN,NaN,NaN,NaN


In [49]:
item_similarity = compute_item_similarity(user_item_matrix)
item_similarity

,item1,item2,item3,item4
item1,1.000000,0.624695,0.6,0.0
item2,0.624695,1.000000,0.0,0.0
item3,0.600000,0.000000,1.0,0.0
item4,0.000000,0.000000,0.0,0.0


In [50]:
pred_item4_user1 = predict_rating_item_based(1, 'item4', user_item_matrix, item_similarity)
print(f"Пользователь 1: предсказанный рейтинг для item4 (холодный старт) = {pred_item4_user1}")

Пользователь 1: предсказанный рейтинг для item4 (холодный старт) = nan


In [51]:
pred_before = predict_rating_item_based(3, 'item1', user_item_matrix, item_similarity)
print(f"Исходное предсказание для пользователя 3 на item1: {pred_before:.3f}")

user_item_matrix.loc[3, 'item2'] = 2.0

item_similarity_new = compute_item_similarity(user_item_matrix)
pred_after = predict_rating_item_based(3, 'item1', user_item_matrix, item_similarity_new)
print(f"Новое предсказание для пользователя 3 на item1 после изменения одной оценки в данных: {pred_after:.3f}")

Исходное предсказание для пользователя 3 на item1: 4.000
Новое предсказание для пользователя 3 на item1 после изменения одной оценки в данных: 2.000
